In [1]:
import fit_tools
from fitparse import FitFile
import pandas as pd
import numpy as np
from bokeh.models import Toggle, BoxAnnotation, CustomJS


In [2]:
fit = FitFile('/Users/jburkhardt/Downloads/2017-08-08-07-28-06.fit')
fit.parse()

df = fit_tools.FitToDataframe(fit, ['power'])


In [3]:
df['power']


timestamp
2017-08-08 11:28:06    220.000000
2017-08-08 11:28:07    194.000000
2017-08-08 11:28:08    184.000000
2017-08-08 11:28:09    215.000000
2017-08-08 11:28:10    175.000000
2017-08-08 11:28:11    173.000000
2017-08-08 11:28:12    162.000000
2017-08-08 11:28:13    157.000000
2017-08-08 11:28:14    164.000000
2017-08-08 11:28:15    127.000000
2017-08-08 11:28:16    126.000000
2017-08-08 11:28:17    126.000000
2017-08-08 11:28:18    148.000000
2017-08-08 11:28:19    118.000000
2017-08-08 11:28:20    119.000000
2017-08-08 11:28:21    143.000000
2017-08-08 11:28:22    135.000000
2017-08-08 11:28:23    118.000000
2017-08-08 11:28:24    116.000000
2017-08-08 11:28:25    113.000000
2017-08-08 11:28:26     85.000000
2017-08-08 11:28:27    139.000000
2017-08-08 11:28:28    182.000000
2017-08-08 11:28:29    157.000000
2017-08-08 11:28:30    153.000000
2017-08-08 11:28:31    141.000000
2017-08-08 11:28:32    144.000000
2017-08-08 11:28:33    172.000000
2017-08-08 11:28:34    169.000000
2017

In [4]:
df = fit_tools.FitPowerCurve(fit)

In [5]:
df.head()

,0
timestamp,
2017-08-08 11:28:06,815.000000
2017-08-08 11:28:07,774.500000
2017-08-08 11:28:08,659.666667
2017-08-08 11:28:09,594.000000
2017-08-08 11:28:10,572.800000


In [6]:
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import HoverTool, BoxZoomTool, ResetTool

# output to static HTML file
output_notebook()

# create a new plot with a title and axis labels
p = figure(title="Power Curve", width=800, height=400, 
           x_axis_type="log", x_axis_label='time',
           y_axis_label='power', tools=[BoxZoomTool(), ResetTool()])

power_data = df[0]
x = [i for i in range(0, len(power_data))]
# add a line renderer with legend and line thickness
p.line(x, power_data, legend="Power", line_width=2)

# show the results
show(p)

Loading BokehJS ...

In [9]:
intervals = fit_tools.FitIntervals(fit)
intervals.FindIntervals(120, 4)
intervals.FindIntervals(6*60, 4)

hover = HoverTool(names=['avg_power'],
    tooltips=[
        ("Power", "@top"),
        ("Heart Rate", "@hr")
    ]
)

# create a new plot with a title and axis labels
p = figure(title="power", width=800, height=240, 
           x_axis_type="datetime", x_axis_label='time', 
           y_axis_label='power', tools=[hover, BoxZoomTool(), ResetTool()])

fdf = intervals.df
power_data = fdf['power']
hr_data = fdf['heart_rate']

# do some smoothing just for prettier rendering
window_size = 60
window = np.ones(window_size)/float(window_size)
power_avg = np.convolve(power_data, window, 'same')

# add a line renderer with legend and line thickness
p.line(fdf.index, power_avg, legend="Power", line_width=2)
p.line(fdf.index, fdf['heart_rate'], color='red', legend="HR", line_width=1)

# convert interval data into ColumnDataSource
idx = 0
left = [i.start for i in intervals.intervals]
top = [int(power_data[i.start:i.end].mean()) for i in intervals.intervals]
right = [i.end for i in intervals.intervals]
bottom = [0 for i in intervals.intervals]
hr = [int(hr_data[i.start:i.end].mean()) for i in intervals.intervals]

source = ColumnDataSource({"top":top, "right":right, "bottom":bottom,"left":left, "hr":hr})
p.quad(top="top", bottom="bottom", right="right", left="left", 
       source=source, color="green", fill_alpha=0.2, name="avg_power")

# show the results
show(p)